In [1]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
warnings.filterwarnings(
    action='ignore',
    category=FutureWarning, 
    module='pyproj'
)

def clean_state_fips_code(
        cellcontents 
):
    """This function takes pandas Series cell contents as input to reformat with leading zeros. 
    Returns modified cell contents.    
    """
    
    # Use conditional to separate cell contents which satisfy the length criteria and those that do not. 
    if len(
        str(
            cellcontents
        )
    ) == 1:
        return format(
            cellcontents, 
            '02'
        )
    else:
        return cellcontents
    
def clean_county_fips_code(
        cellcontents
):
    """This function takes pandas Series cell contents as input to reformat with leading zeros. 
    Returns modified cell contents.    
    """
    
    if len(
        str(
            cellcontents
        )
    ) == 1:
        
        return format(
            cellcontents, 
            '03'
        )
    
    elif len(
        str(
            cellcontents
        )
    ) == 2:
        
        return format(
            cellcontents, 
            '03'
        )
    
    else:
        
        return cellcontents
    
def munge():
    
    """Function to munge county-level case data with population, hospital, and income information.
    Returns a pandas DataFrame for use in plotly. 
    """
    
    # Remote data sources. To be updated to covid-19-us-api for those not already updated
    county_API_URL = "https://covid19-us-api.herokuapp.com/county"
    hospitals_url = "https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.geojson"
    counties_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
    population_url = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2018/counties/totals/co-est2018-alldata.csv"

    # Local data to be uploaded as covid-19-us-api 
    population_path = "/home/alex/Downloads/co-est2018-alldata.csv" #local copy of population_url
    income_path = "/home/alex/data/Unemployment.csv" #local copy of census income data
    
    # Load County API data and convert to geopandas GeoDataFrame of point geometry features
    county_API_data = requests.get(
        county_API_URL
    ).json()["message"]
    
    county_API_df = pd.read_json(
        county_API_data, 
        orient="records"
    ) 
    
    county_API_gdf = gpd.GeoDataFrame(
        county_API_df, 
        geometry=gpd.points_from_xy(
            county_API_df.Longitude, 
            county_API_df.Latitude
        )
    )
    
    # Define coordinate reference system of Latitude, Longitude data
    county_API_gdf.crs = "EPSG:4326"
    
    # Ingest US County Boundary data from plotly dash github repository
    us_county_boundaries_gdf = gpd.read_file(
        counties_url
    )
    
    # Reproject county boundary data to same projection as county API data
    us_county_boundaries_gdf = us_county_boundaries_gdf.to_crs(
        "EPSG:4326"
    )
        
    # Perform spatial join to intersect county boundary geometries and county api points, applying FIPS labels to API data lacking this information
    county_API_with_FIPS = gpd.sjoin(
        us_county_boundaries_gdf,
        county_API_gdf,
        how="left"
    ).drop(
        [
            "geometry",
             "GEO_ID",
             "COUNTY",
             "LSAD",
             "CENSUSAREA",
             "index_right"
        ], 
        axis=1
    )
    
    # Ingest hospital locations database as geopandas GeoDataFrame
    hospitals_gdf = gpd.read_file(
        hospitals_url
    )
    
    # Subset for open hospitals
    hospitals_gdf = hospitals_gdf[
        hospitals_gdf[
            "STATUS"
        ] == "OPEN"
    ]
    
    # Drop geometry column and convert to pandas DataFrame
    hospitals_df = hospitals_gdf.drop(
        [
            "geometry", 
            "STATUS"
        ], 
        axis=1
    )
    
    # Clean BEDS Series by replacing NaN placeholder
    hospitals_df[
        "BEDS"
    ] = hospitals_df[
        "BEDS"
    ].replace(
        -999, 
        np.NaN
    )
    
    # Join County API (with FIPS labels) with the hospital locations using FIPS attributes
    county_API_with_FIPS_and_Hospitals = county_API_with_FIPS.merge(
        hospitals_df, 
        left_on="id", 
        right_on="COUNTYFIPS"
    )
    
    # Count the number of hospitals in each county then create new associated column in County API data (with FIPS labels)
    county_API_with_FIPS[
        "Hospital Count"
    ] = county_API_with_FIPS[
        "id"
    ].map(
        county_API_with_FIPS_and_Hospitals.groupby(
            "COUNTYFIPS"
        )[
            "ID"
        ].count().copy().to_dict()
    )
    
    # Repeat the same procedure but sum the number of hospital beds in each county
    county_API_with_FIPS[
        "Bed Count"
    ] = county_API_with_FIPS[
        "id"
    ].map(
        county_API_with_FIPS_and_Hospitals.groupby(
            "COUNTYFIPS"
        )[
            "BEDS"
        ].sum().copy().to_dict()
    )
    
    # Convert identifier column to integer for further data joining
    county_API_with_FIPS[
        "id"
    ] = county_API_with_FIPS[
        "id"
    ].astype(int)
    
    # Ingest population data 
    population_df = pd.read_csv(
        population_path, 
        engine="python"
    )
    
    # Create intermediate state-level FIPS identifier column using convenience function
    population_df[
        "STATE_FC"
    ] = population_df[
        "STATE"
    ].apply(
        clean_state_fips_code
    )
    
    # Create intermediate county-level FIPS identifier column using convenience function
    population_df[
        "COUNTY_FC"
    ] = population_df[
        "COUNTY"
    ].apply(
        clean_county_fips_code
    )
    
    # Generate integer county-level FIPS identifier from intermediate state- and county-level columns 
    population_df[
        "FIPS_CODE"
    ] = (population_df[
        "STATE"
    ].astype(
        str
    ) + population_df[
        "COUNTY_FC"
    ].astype(
        str
    )).astype(
        int
    )
    
    # Create a county-level FIPS identifier for use with plotly Mapbox choropleth module
    population_df[
        "FIPS_plotly"
    ] = population_df[
        "STATE_FC"
    ].astype(
        str
    ) + population_df[
        "COUNTY_FC"
    ].astype(
        str
    )
    
    # Subset population data for columns of interest, namely FIPS identifiers and 2018 population estimates
    population_df = population_df[
        [
            "FIPS_CODE",
             "FIPS_plotly",
             "POPESTIMATE2018"
        ]
    ]
    
    # Join County API data (with FIPS labels) with population data 
    county_data = county_API_with_FIPS.merge(
        population_df,
        left_on="id",
        right_on="FIPS_CODE",
        how="left"
    )
    
    # Calculate percent of county population confirmed
    county_data[
        "Percent of Population Confirmed"
    ] = (
        county_data[
            "Confirmed"
        ] / county_data[
            "POPESTIMATE2018"
        ]
    )*100
    

    # Calculate number of hospitals per ten thousand people in each US county
    county_data[
        "Hospitals per ten thousand people"
    ] = (
        county_data[
            "Hospital Count"
        ] / county_data[
            "POPESTIMATE2018"
        ] * 10000
    )
    
    # Calculate number of hospital beds per ten thousand people in each US county
    county_data[
        "Hospital beds per ten thousand people"
    ] = (
        county_data[
            "Bed Count"
        ] / county_data[
            "POPESTIMATE2018"
        ] * 10000
    )
    
    # Drop duplicate FIPS-code counties
    county_data = county_data.drop_duplicates(
        "id"
    )
    
    # Ingest income data 
    income_df = pd.read_csv(
        income_path,
        skiprows=4
    )
    # Clean income column by removing dollar signs and commas then converting to float type
    income_df[
        "Median_Household_Income_2018"
    ] = income_df[
        "Median_Household_Income_2018"
    ].str.replace(
        "$",
        ""
    ).str.replace(
        ",",
        ""
    ).astype(
        float
    )
    
    # Merge county api (with labels) data with income data
    county_data_with_income = county_data.merge(
        income_df[
            [
                "FIPS",
                "Median_Household_Income_2018"
            ]
        ],
        left_on="id",
        right_on="FIPS",
        how="left"
    )
    
    # Drop duplicate columns in final dataframe (based on FIPS ids)
    county_data_with_income = county_data_with_income.drop_duplicates(
        "id"
    )
    
    # Clean fatality rate column by removing percent sign then converting to float type
    county_data_with_income[
        "Fatality Rate"
    ] = county_data_with_income[
        "Fatality Rate"
    ].str.replace(
        "%",
        ""
    ).astype(
        float
    )
    
    # Return pandas DataFrame of desired results
    return county_data_with_income

In [2]:
# Run function and save into object called df
df = munge()

# Print the shape of df
print(df.shape)

# And return first five rows of df
df.head()

(3221, 23)


,id,STATE,NAME,County Name,State Name,Confirmed,New,Death,New Death,Fatality Rate,...,Hospital Count,Bed Count,FIPS_CODE,FIPS_plotly,POPESTIMATE2018,Percent of Population Confirmed,Hospitals per ten thousand people,Hospital beds per ten thousand people,FIPS,Median_Household_Income_2018
0,1001,01,Autauga,Autauga,Alabama,7.0,1.0,0.0,0.0,0.0,...,1.0,85.0,1001.0,01001,55601.0,0.012590,0.179853,15.287495,1001.0,59338.0
1,1009,01,Blount,Blount,Alabama,5.0,0.0,0.0,0.0,0.0,...,1.0,40.0,1009.0,01009,57840.0,0.008645,0.172891,6.915629,1009.0,50412.0
2,1017,01,Chambers,Chambers,Alabama,36.0,9.0,4.0,4.0,11.1,...,1.0,115.0,1017.0,01017,33615.0,0.107095,0.297486,34.210918,1017.0,39917.0
3,1021,01,Chilton,Chilton,Alabama,10.0,2.0,0.0,0.0,0.0,...,1.0,26.0,1021.0,01021,44153.0,0.022649,0.226485,5.888615,1021.0,47547.0
4,1033,01,Colbert,Colbert,Alabama,4.0,2.0,0.0,0.0,0.0,...,2.0,363.0,1033.0,01033,54762.0,0.007304,0.365217,66.286841,1033.0,49055.0


In [3]:
# Return last five rows of df
df.tail()

,id,STATE,NAME,County Name,State Name,Confirmed,New,Death,New Death,Fatality Rate,...,Hospital Count,Bed Count,FIPS_CODE,FIPS_plotly,POPESTIMATE2018,Percent of Population Confirmed,Hospitals per ten thousand people,Hospital beds per ten thousand people,FIPS,Median_Household_Income_2018
3216,51001,51,Accomack,Accomack,Virginia,6.0,3.0,0.0,0.0,0.0,...,NaN,NaN,51001.0,51001,32412.0,0.018512,NaN,NaN,51001.0,42879.0
3217,51021,51,Bland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,51021.0,51021,6293.0,NaN,NaN,NaN,51021.0,47681.0
3218,51027,51,Buchanan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,134.0,51027.0,51027,21221.0,NaN,0.471231,63.144998,51027.0,32144.0
3219,51037,51,Charlotte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,51037.0,51037,11938.0,NaN,NaN,NaN,51037.0,41382.0
3220,51041,51,Chesterfield,Chesterfield,Virginia,21.0,9.0,0.0,0.0,0.0,...,4.0,466.0,51041.0,51041,348556.0,0.006025,0.114759,13.369444,51041.0,80734.0
